## <center>Analiza poziomu PM2.5 w afrykańskich miastach</center>
### Zespół:
<ol>
    <li style='font-size: 20px'>Hubert Kłosowski 242424</li>
    <li style='font-size: 20px'>Krzysztof Kolanek 242425</li>
    <li style='font-size: 20px'>Kamil Małecki 242464</li>
</ol>

### Potrzebne importy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Wczytanie danych

In [ ]:
train = pd.read_csv('data\\train.csv')
test = pd.read_csv('data\\test.csv')

train.info()

In [ ]:
train.head()

### Rozbicie daty na składowe

In [ ]:
def change_date(dataframe):
    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe['day'] = dataframe['date'].dt.dayofweek.astype('category')
    dataframe['month'] = dataframe['month'].astype('category')
    dataframe['hour'] = dataframe['hour'].astype('category')
    return dataframe


train, test = change_date(train), change_date(test)

### Wykres przedstawiający jakość powietrza w krajach afrykańskich

In [ ]:
sns.lineplot(data=train, x='date', y='pm2_5', hue='country')
plt.title('Jakość powietrza z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 w zarejestrowanych godzinach

In [ ]:
sns.barplot(data=train, x='hour', y='pm2_5', hue='country')
plt.title('Jakość powietrza w poszczególnych godzinach z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 z zależności od dnia tygodnia

In [ ]:
sns.barplot(data=train, x='day', y='pm2_5', hue='country')
plt.title('Jakość powietrza w każdym dniu tygodnia z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 z zależności od miesiąca

In [ ]:
sns.barplot(data=train, x='month', y='pm2_5', hue='country')
plt.title('Jakość powietrza w każdym dniu tygodnia z podziałem na kraje')

### Korelacja wybranych kolumn z pm2_5

In [ ]:
sns.heatmap(train[['month', 'day', 'hour', 'site_latitude', 'site_longitude', 'cloud_surface_albedo', 'pm2_5']].corr(), annot=True, cmap='Greys')

## <center>Czyszczenie danych</center>

### 1. Uzupełnienie wartości brakujących

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer


def fill_train(column_name='site_latitude'):
    column_values = X[column_name].unique()
    for date in column_values:
        for i, column in enumerate(starts_with):
            similar_columns = [col for col in X.columns if col.startswith(column)]
            df = X.loc[X[column_name] == date, similar_columns].copy()
            if not df.empty:
                try:
                    X.loc[X[column_name] == date, similar_columns] = imputers[i].fit_transform(df, y)
                except ValueError:
                    X.drop(index=df.index, inplace=True)
                    y.drop(index=df.index, inplace=True)
                    X.reset_index(drop=True, inplace=True)
                    y.reset_index(drop=True, inplace=True)

def fill_test(column_name='site_latitude'):
    column_values = test[column_name].unique()
    for date in column_values:
        for i, column in enumerate(starts_with):
            similar_columns = [col for col in test.columns if col.startswith(column)]
            df = test.loc[test[column_name] == date, similar_columns].copy()
            if not df.empty:
                test.loc[test[column_name] == date, similar_columns] = imputers[i].transform(df)

def drop_high_nans(dataframe):  # usuwamy kolumny o dużej liczbie wartości NaN
    columns_nans = []
    for i, el in enumerate(dataframe.columns):
        if dataframe[el].isna().sum() / len(dataframe) >= 0.9:
            columns_nans.append(el)
    dataframe.drop(columns_nans, axis=1, inplace=True)
    return dataframe

def drop_high_correlated_columns():
    matrix = X.corr(numeric_only=True).abs()
    upper_t = matrix.where(np.triu(np.ones_like(matrix, dtype=np.bool_), k=1))
    return [col for col in upper_t.columns if any(upper_t[col] > 0.99)]


test_ids = test['id']
train.drop(columns=['id', 'city', 'country', 'site_id', 'date'], inplace=True)
test.drop(columns=['id', 'city', 'country', 'site_id', 'date'], inplace=True)
starts_with = train.columns.str.split('_', expand=True).levels[0].to_frame()
starts_with.drop(['month', 'day', 'hour', 'pm2'], inplace=True)
starts_with = starts_with[0].tolist()

imputers = [KNNImputer(n_neighbors=15, weights='distance') for _ in range(len(starts_with))]
# imputers = [IterativeImputer() for _ in range(len(starts_with))]
X, y = train.drop(['pm2_5'], axis=1), train['pm2_5']
X, test = drop_high_nans(X), drop_high_nans(test)
# fill_train(), fill_test()
# to_drop = drop_high_correlated_columns()
# X, test = X.drop(columns=to_drop, axis=1), test.drop(columns=to_drop, axis=1)

### Wykresy pudełkowe wskazujące wartości odstające

In [ ]:
from sympy import divisors


def plot_boxplots():
    for i, column_group in enumerate(starts_with):
        similar_columns = [col for col in train.columns if col.startswith(column_group)]
        if len(similar_columns) > 1:
            divs = divisors(len(similar_columns))
            if len(divs) % 2 == 0:
                rows, cols = divs[(len(divs) // 2) - 1], divs[len(divs) // 2]
            else:
                rows, cols = divs[len(divs) // 2], divs[len(divs) // 2]
            fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(40, 30), squeeze=False)
            fig.suptitle(column_group, fontsize=25)
            for j, column in enumerate(similar_columns):
                x_cord, y_cord = divmod(j, cols)
                train[column].plot(kind='box', ax=ax[x_cord, y_cord], fontsize=15)
                if column in ver.keys():
                    ax[x_cord, y_cord].axhline(y=ver.get(column)[0] * train[column].std(), color='green', label='down-limit')
                    ax[x_cord, y_cord].axhline(y=ver.get(column)[1] * train[column].std(), color='red', label='up-limit')
                    ax[x_cord, y_cord].legend(loc='best')
            plt.show()


vertical_columns = [col for col in X.columns if 'number_density' in col]
ver = dict(zip(vertical_columns, [(-5 * X[column].std(), 5 * X[column].std()) for column in vertical_columns]))

# plot_boxplots()

### 2. Usunięcie wartości odstających

In [ ]:
from scipy.stats import zscore


def del_outliers():
    zscores = zscore(X.select_dtypes(exclude='category').values, nan_policy='omit')
    np.nan_to_num(zscores, copy=False)
    zscores = np.absolute(zscores)
    result = np.mean(zscores, axis=1)
    indexes_to_drop = []
    q1, q2 = np.quantile(result, 0.005), np.quantile(result, 0.995)
    for i, el in enumerate(result):
        if q1 <= el >= q2:
            indexes_to_drop.append(i)
    X.drop(indexes_to_drop, inplace=True)
    y.drop(indexes_to_drop, inplace=True)
    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)


# del_outliers()

X.info()

In [ ]:
X.head()

## <center>Selekcja cech</center>

In [ ]:
from sklearn.feature_selection import RFECV, RFE, SelectKBest, mutual_info_regression, f_regression
from sklearn.ensemble import RandomForestRegressor


def plot_feature_importance(sc, num_of_features):
    if isinstance(sc, RFECV) or isinstance(sc, RFE):
        scores = dict(zip(sc.feature_names_in_, sc.ranking_))
    else:
        scores = dict(zip(sc.feature_names_in_, sc.scores_))
    scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:num_of_features]
    scores_df = pd.DataFrame(scores, columns=['Feature', 'Score'])
    
    scores_df.plot(kind='bar', x='Feature', y='Score', figsize=(10, 6), rot=90, title='Oceny wybranych cech')
    plt.xlabel('Cecha')
    plt.ylabel('Ocena')


# selector = RFE(
#     estimator=RandomForestRegressor(
#         n_estimators=700, 
#         max_depth=7, 
#         random_state=4, 
#         n_jobs=-1, 
#         oob_score=True,
#         warm_start=True
#     ),
#     n_features_to_select=k,
# )
# k = 17
# selector = RFECV(
#     estimator=RandomForestRegressor(
#         n_estimators=400, 
#         max_depth=10, 
#         random_state=4, 
#         n_jobs=-1, 
#         oob_score=True, 
#         warm_start=True, 
#         ccp_alpha=1e-4
#     ),
#     min_features_to_select=k, 
#     cv=10, 
#     scoring='neg_root_mean_squared_error',
#     n_jobs=-1
# )
# selector.fit(X, y)
# X, test = selector.transform(X), selector.transform(test)
# 
# plot_feature_importance(selector, k)

## <center>Transformacja danych</center>

### 1. Standaryzacja danych

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

# Bez kategorycznych
# scale_columns = X.columns.difference(['hour', 'month', 'day'])
# 
# scaler = make_column_transformer((StandardScaler(), scale_columns))
# 
# X_cat, test_cat = X[['hour', 'month', 'day']], test[['hour', 'month', 'day']]
# 
# X = pd.concat([pd.DataFrame(scaler.fit_transform(X[scale_columns]), columns=scaler.feature_names_in_), X_cat], axis=1)
# test = pd.concat([pd.DataFrame(scaler.transform(test[scale_columns]), columns=scaler.feature_names_in_), test_cat], axis=1)
# Wszyskie kolumny
scaler = StandardScaler()

X = scaler.fit_transform(X, y)
X = pd.DataFrame(X, columns=scaler.feature_names_in_)
test = scaler.transform(test)

### 2. Podział na zbiór walidacyjny i treningowy

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

## <center>Część obliczeniowa</center>

### Otrzymanie najlepszych parametrów

In [ ]:
def save_to_csv(y_pred, save_as):
    final_df = pd.concat([test_ids, pd.DataFrame.from_dict({'pm2_5': y_pred})], axis=1)
    final_df.to_csv(f'result\\{save_as}', index=False)

### Wybór karty graficznej dla PyTorcha

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader


# device = (
#     'cuda'
#     if torch.cuda.is_available()
#     else 'mps'
#     if torch.backends.mps.is_available()
#     else 'cpu'
# )
# 
# X_train_tensor = torch.tensor(X_train, device=device, dtype=torch.float)
# X_test_tensor = torch.tensor(X_test, device=device, dtype=torch.float)
# y_train_tensor = torch.tensor(y_train.to_numpy(), device=device, dtype=torch.float)
# y_test_tensor = torch.tensor(y_test.to_numpy(), device=device, dtype=torch.float)
# test_tensor = torch.tensor(test, device=device, dtype=torch.float)

### Optuna + Pytorch

In [ ]:
# def rmse_loss(y_true, y_pred):
#     return torch.sqrt(torch.mean((y_true - y_pred) ** 2))
# 
# def define_model(trial):
#     n_layers = trial.suggest_int('n_layers', 2, 5)
#     layers = []
#     
#     in_features = X_train_tensor.shape[1]
#     for i in range(n_layers):
#         out_features = trial.suggest_int(f'layer{i}', 25, 200)
#         layers.append(nn.Linear(in_features, out_features))
#         layers.append(getattr(torch.nn, trial.suggest_categorical(f'nn_{i}', ['ReLU', 'Sigmoid', 'Tanh']))())
#         layers.append(nn.Dropout(trial.suggest_float(f'dropout{i}', 0.1, 0.6)))
#         in_features = out_features
#     layers.append(nn.Linear(in_features, 1))
#     return nn.Sequential(*layers)
# 
# def objective(trial):
#     model = define_model(trial).to(device=device)
#     lr = trial.suggest_float('lr', 1e-4, 1e-2)
#     weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-4)
#     betas = (trial.suggest_float('beta1', 0.8, 0.9), trial.suggest_float('beta2', 0.997, 0.999))
#     optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay, betas=betas, fused=True)
#     
#     batch_size = trial.suggest_int('batch_size', 16, 416, step=25)
#     criterion = rmse_loss
#     epochs = trial.suggest_int('epochs', 100, 300, step=50)
#     
#     final_train_tensor = torch.concat((X_train_tensor, y_train_tensor.unsqueeze(dim=1)), dim=1)
#     dataset = DataLoader(final_train_tensor, batch_size=batch_size, shuffle=True)
#     
#     model.train()
#     
#     for epoch in range(epochs):
#         epoch_loss = 0
#         for batch_idx, batch in enumerate(dataset):
#             inputs, targets = batch[:, :-1], batch[:, -1]
#             batch_pred = model(inputs)
#             optimizer.zero_grad()
#             loss = criterion(targets, batch_pred.squeeze())
#             epoch_loss += loss.item()
#             loss.backward()
#             optimizer.step()
#             
#         trial.report(epoch_loss / len(dataset), epoch)
#         if trial.should_prune():
#             raise optuna.exceptions.TrialPruned()
#         
#     model.eval()
#     
#     with torch.no_grad():
#         y_pred = model(X_test_tensor).squeeze()
#         
#     return root_mean_squared_error(y_test, y_pred.numpy(force=True))
# 
# study = optuna.create_study(
#     direction='minimize',
#     sampler=optuna.samplers.TPESampler(),
#     study_name='AirQuality',
# )
# study.optimize(objective, n_trials=200)
# model_params = ['lr', 'n_layers', 'weight_decay', 'beta1', 'beta2', 'epochs', 'batch_size']

### <center>Optuna + lightGBM</center>

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.metrics import root_mean_squared_error

def define_lightgbm_model(trial):
    params = {
        'objective': 'root_mean_squared_error',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 50),
        'max_depth': trial.suggest_int('max_depth', 15, 25),
        'learning_rate': trial.suggest_float('learning_rate', 8e-3, 5e-2),
        'n_estimators': trial.suggest_int('n_estimators', 900, 1100),
        'tree_learner': 'voting',
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 0.9),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 50, 150),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 1e-8, 1, log=True),
        'bagging_freq': 1,
        'device': 'cpu',
        'n_jobs': -1,
        'random_state': 4,
        'verbosity': -1
    }
    return lgb.LGBMRegressor(**params)

def objective_lightgbm(trial):
    model = define_lightgbm_model(trial)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return root_mean_squared_error(y_test, y_pred)

study = optuna.create_study(direction='minimize', study_name='AirQualityWithLightGBM', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(objective_lightgbm, n_trials=500)

### Zdefiniowanie najlepszego lightgbm

In [ ]:
params_12 =  {'num_leaves': 25, 
           'max_depth': 15, 
           'learning_rate': 0.01982093884782807, 
           'n_estimators': 1042, 
           'tree_learner': 'voting', 
           'bagging_fraction': 0.863457680863147, 
           'subsample': 0.8572357579881347, 
           'colsample_bytree': 0.8692866219741755, 
           'min_data_in_leaf': 57,
           'bagging_freq': 1,
            'device': 'cpu',
            'n_jobs': -1,
            'random_state': 4,
            'verbosity': -1,
           'objective': 'root_mean_squared_error',
            'boosting_type': 'gbdt',
           }
# lgbm = lgb.LGBMRegressor(**params_12)
lgbm = define_lightgbm_model(study.best_trial)
lgbm.fit(X_train, y_train)
model_params = ['num_leaves', 'max_depth', 'learning_rate', 'n_estimators', 'subsample', 'colsample_bytree', 'min_data_in_leaf', 'bagging_fraction']

In [ ]:
root_mean_squared_error(y_test, lgbm.predict(X_test))

### Wykres przedstawiający każdy <i>trial</i> w procesie nauki

In [ ]:
optuna.visualization.plot_optimization_history(study)

### Wizualizacja przekroju parametrów

In [ ]:
optuna.visualization.plot_slice(study, params=model_params)

### Wpływ poszczególnych parametrów na proces nauki modelu

In [ ]:
optuna.visualization.plot_param_importances(study)

### Najlepsze parametry

In [ ]:
study.best_params

### Znaczenie poszczególnych kolumn

In [ ]:
lgb.plot_importance(lgbm, figsize=(20, 12), dpi=200)

### Drzewo decyzyjne

In [ ]:
lgb.plot_tree(lgbm, precision=2, figsize=(20, 12), show_info=['data_percentage'], dpi=200, orientation='vertical')

### <center>Optuna + RandomForestRegressor</center>

In [ ]:
def define_rfr_model(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 900),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'min_samples_split': trial.suggest_int('min_samples_split', 10, 50),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 50),
        'bootstrap': trial.suggest_categorical('bootstrap', [True]),
        'oob_score': trial.suggest_categorical('oob_score', [True, False]),
        'warm_start': trial.suggest_categorical('warm_start', [True, False]),
        'random_state': 4,
        'n_jobs': -1,
    }
    return RandomForestRegressor(**params)

def objective_rfr(trial):
    model = define_rfr_model(trial)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return root_mean_squared_error(y_test, y_pred)

study = optuna.create_study(direction='minimize', study_name='AirQualityWithRandomForestRegressor', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(objective_rfr, n_trials=200)

### Zdefiniowanie najlepszego RandomForestRegressor

In [ ]:
rfr = define_rfr_model(study.best_trial)
rfr.fit(X_train, y_train)

In [ ]:
root_mean_squared_error(y_test, rfr.predict(X_test))

## <center>Do wysłania</center>

### Pytorch

In [ ]:
# best_model = define_model(study.best_trial)
# best_model.to(device=device)
# 
# best_model.eval()
# 
# with torch.no_grad():
#     save_to_csv(best_model(test_tensor).squeeze().numpy(force=True), 'nn.csv')

### lightGBM

In [ ]:
save_to_csv(lgbm.predict(test), 'lightgbm.csv')

### RandomForestRegressor

In [ ]:
save_to_csv(rfr.predict(test), 'rfr.csv')

### Dodatkowe informacje
<ol>
    <li>The 15km SO2 band is ingested only when solar_zenith_angle < 70.</li>
    <li>Because of noise on the data, negative vertical column values are often observed in particular over clean regions or for low SO2 emissions. It is recommended not to filter these values except for outliers, i.e. for vertical columns lower than -0.001 mol/m^2.</li>
    <li>The effective cloud fraction is the radiometric equivalent cloud fraction of a satellite pixel assuming a fixed cloud albedo, usually 0.8. By definition the effective cloud fraction times the assumed cloud albedo plus the cloud-free surface and atmosphere contributions yields a TOA reflectance that agrees with the observed TOA reflectance.</li>
</ol>